In [9]:
!pip install langchain openai huggingface_hub -q

In [10]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = OpenAI(model_name = "text-davinci-003",
             temperature = 0,
             max_tokens = 256)

In [14]:
article = """It may well be another 24 hours or more before the 40 workers trapped since November 12 inside a collapsed portion of an under-construction tunnel on the Char Dham highway project in Uttarakhand can be pulled out, officials have said amidst the ongoing race against time to successfully complete the rescue mission.

Boring machines from Dehradun and a team of eight experts are at the site where a portion of the upcoming 4.5-km tunnel between Silkyara and Barkot on the Yamunotri-Gangotri national highway collapsed due to landslide, leaving the workers trapped. The Rs 853-crore tunnel is part of the contentious Char Dham all-weather road project. The roof of a 100-metre stretch caved in at around 5.30 am on November 12 while workers were carrying out re-profiling work, a statement from the ministry of road transport and highways (MoRTH) said.

Uttarakhand State Disaster Management Authority officials said special machines had been deployed to bore a pipe into the dense rubble so that the trapped workers can be evacuated through it, adding that the rescue operation was a tough challenge in itself thanks to the condition of the soil at the mishap site. “This is the only way. The soil there is loose. The boring will have to be done with extreme care or else the rescue machines might also collapse,” Dr Ranjit Kumar Sinha, secretary, disaster management, Uttarakhand, told INDIA TODAY.

Officials of the National Highways & Infrastructure Development Corporation Limited (NHIDCL), under MoRTH, claimed the trapped workers had ample space inside to move around and had been provided food. “The impression that they are somehow trapped under the debris is not true. The roof collapsed midway. Thankfully, there is around 2 km worth of open space inside on the blind side of the tunnel for them move around,” said a senior ministry official.

There is electricity and contact has been established with the workers through walkie-talkie. “They (trapped workers) said they were fine but needed food. We have been able to insert a pressure pipe of 4-inch diameter to supply dry fruits to them,” the official said. In multiple posts on social media platform X on November 13, the NHIDCL said mobilisation of men, material and machinery to push a 900-mm (35 inches) diameter steel pipe with the help of hydraulic jack for evacuation of workers, was in process. “Experts from RVNL (Rail Vikas Nigam Limited), RITES (Rail India Technical and Economic Service), [Uttarakhand State Disaster Management] Authority’s engineers, multi-agencies such as SDRF (State Disaster Response Force), NDRF (National Disaster Response Force), and other senior officials rushed to the site for close monitoring of the rescue operation. We want to assure that all necessary measures are being taken and adequate machinery has been deployed,” the NHIDCL said in a post.

MoRTH did not commit to when the rescue would conclude. “All coordinated efforts are being made to evacuate the stranded workforce at the earliest,” the ministry’s statement said. It added that the construction of the tunnel would “immensely benefit pilgrims” as it would provide all-weather connectivity and reduce travel across a 25.6 km snow-affected length of NH-134 (Dharasu-Barkot-Yamunotri road) to 4.5 km, cutting travel time from 50 minutes to five.

NHIDCL contractor Navyug Engineering Company had begun constructing the tunnel in July 2018 with a target to complete it by July 2022. “The work is delayed. With the present progress of work at 56 per cent, it was likely to be completed by May 14, 2024,” the ministry statement said.

This is yet another landslide-related mishap to hit the Rs 12,000-crore Char Dham Highway project, essentially a road-widening exercise, which has been mired in controversy ever since it was sanctioned in 2016. During monsoon this year, another leg of the project, the Badrinath highway, remained closed for days thanks to portions collapsing.

Locals as well as environmentalists have been protesting against the massive construction exercise, claiming it is destabilising the fragile mountain ecosystem—a charge the Union government has refuted repeatedly.

The 850-km Char Dham project seeks to connect the four Hindu shrines of Gangotri, Yamunotri, Kedarnath and Badrinath through a network of all-weather highways, besides creating roads of strategic importance for faster movement of troops and military equipment to the border with China."""

len(article)

4445

In [15]:
fact_extraction_prompt = PromptTemplate(
    input_variables=['text_input'],
    template="Extract the key facts out of this article. Don't include opinion. Give each fact a number and keep them short: \n\n {text_input}"
)

In [17]:
chain_fact_extraction = LLMChain(
    llm = llm,
    prompt = fact_extraction_prompt,

)

facts = chain_fact_extraction.run(article)
print(facts)



Facts: 
1. 40 workers have been trapped since November 12 in a collapsed portion of an under-construction tunnel on the Char Dham highway project in Uttarakhand. 
2. Boring machines from Dehradun and a team of eight experts are at the site. 
3. The Rs 853-crore tunnel is part of the contentious Char Dham all-weather road project. 
4. Special machines have been deployed to bore a pipe into the dense rubble so that the trapped workers can be evacuated through it. 
5. The trapped workers have ample space inside to move around and have been provided food. 
6. Electricity and contact has been established with the workers through walkie-talkie. 
7. The NHIDCL contractor Navyug Engineering Company had begun constructing the tunnel in July 2018 with a target to complete it by July 2022. 
8. Locals and environmentalists have been protesting against the massive construction exercise. 
9. The 850-km Char Dham project seeks to connect the four Hindu shrines of Gangotri, Yamunotri, Kedarnath and 

# Rewriteas a Summary from the facts

In [18]:
summary_prompt = PromptTemplate(
    input_variables=['facts'],
    template = """Your task is to utilize following list of facts to a short paragraph. Make sure you're not missing any facts.
    Facts: {facts}"""
)

In [19]:
summary_chain = LLMChain(
    llm = llm,
    prompt = summary_prompt
)

summary = summary_chain.run(facts)
print(summary)



Since November 12, 40 workers have been trapped in a collapsed portion of an under-construction tunnel on the Char Dham highway project in Uttarakhand. Boring machines from Dehradun and a team of eight experts have been deployed to the site to bore a pipe into the dense rubble so that the trapped workers can be evacuated. The Rs 853-crore tunnel is part of the contentious Char Dham all-weather road project, which seeks to connect the four Hindu shrines of Gangotri, Yamunotri, Kedarnath and Badrinath through a network of all-weather highways. The trapped workers have ample space inside to move around and have been provided food, and electricity and contact has been established with the workers through walkie-talkie. Locals and environmentalists have been protesting against the massive construction exercise, which was begun by the NHIDCL contractor Navyug Engineering Company in July 2018 with a target to complete it by July 2022.


# Using facts to create triples

In [21]:
triples_prompt = PromptTemplate(
    input_variables=['facts'],
    template = "Take the following list of facts and turn it into triples for a knowledge graph. \n\n {facts}"
)

In [22]:
triples_chain = LLMChain(
    llm = llm,
    prompt = triples_prompt
)

triples = triples_chain.run(facts)
print(triples)
print(len(triples))



Triples: 
1. (Char Dham Highway Project, located in, Uttarakhand) 
2. (Char Dham Highway Project, has tunnel, collapsed portion) 
3. (Char Dham Highway Project, has contractor, Navyug Engineering Company) 
4. (Char Dham Highway Project, has target completion date, July 2022) 
5. (Collapsed portion, has trapped workers, 40) 
6. (Collapsed portion, has experts, 8) 
7. (Collapsed portion, has machines, boring machines from Dehradun) 
8. (Collapsed portion, has evacuation method, pipe) 
9. (Collapsed portion, has food, yes) 
10. (Collapsed portion, has electricity, yes) 
11. (Collapsed portion, has communication, walkie-talkie) 
12. (Char Dham Project, connects, four Hindu shrines) 
13. (Char Dham Project, has protesters, locals and environmentalists)
759


# Chains above chains together

In [23]:
from langchain.chains import SequentialChain, SimpleSequentialChain

chain = SimpleSequentialChain(
    chains = [chain_fact_extraction, summary_chain, triples_chain],
    verbose = True
)

In [24]:
response = chain.run(article)
print(response)



> Entering new SimpleSequentialChain chain...


Facts: 
1. 40 workers have been trapped since November 12 in a collapsed portion of an under-construction tunnel on the Char Dham highway project in Uttarakhand. 
2. Boring machines from Dehradun and a team of eight experts are at the site. 
3. The Rs 853-crore tunnel is part of the contentious Char Dham all-weather road project. 
4. A 100-metre stretch caved in at around 5.30 am on November 12 while workers were carrying out re-profiling work. 
5. Special machines have been deployed to bore a pipe into the dense rubble so that the trapped workers can be evacuated through it. 
6. The trapped workers have ample space inside to move around and have been provided food. 
7. There is electricity and contact has been established with the workers through walkie-talkie. 
8. Mobilisation of men, material and machinery to push a 900-mm (35 inches) diameter steel pipe with the help of hydraulic jack for evacuation of workers is in process. 
9. Exp

# Program-Aided Language (PAL) Model - Math chain
uses LLMs to read natural language problems and generate programs as reasoning steps

In [ ]:
!pip install langchain_experimental

In [32]:
from langchain_experimental.pal_chain.math_prompt import MATH_PROMPT

In [34]:
from langchain_experimental.pal_chain.base import PALChain

pal_chain = PALChain.from_math_prompt(llm, verbose = True)

In [42]:
question_1 = "How much time will it take for Raghav to reach the top of the 100-foot iron rod if he climbs 10 feet during the day but descends by 5 feet at night?"
question_2 = "Jill has 4 apple, 3 mango and 6 orange. The cost of one mango is twice of orange and 1.5 time of apple. The cost of each mango is 60. How many apples will Jill be having if he exchanged all of this orange and mangoes to buy apples?"

In [38]:
pal_chain.run(question_1)



> Entering new PALChain chain...


def solution():
    """How much time will it take for Raghav to reach the top of the 100-foot iron rod if he climbs 10 feet during the day but descends by 5 feet at night?"""
    rod_height = 100
    climb_rate = 10
    descent_rate = 5
    total_distance = rod_height
    total_time = total_distance / (climb_rate - descent_rate)
    result = total_time
    return result

> Finished chain.


'20.0'

In [49]:
pal_chain.run(question_2)



> Entering new PALChain chain...
def solution():
    """Jill has 4 apple, 3 mango and 6 orange. The cost of one mango is twice of orange and 1.5 time of apple. The cost of each mango is 60. How many apples will Jill be having if he exchanged all of this orange and mangoes to buy apples?"""
    apples_initial = 4
    mangoes = 3
    oranges = 6
    mango_cost = 60
    orange_cost = mango_cost / 2
    apple_cost = mango_cost / 1.5
    total_cost = (mangoes * mango_cost) + (oranges * orange_cost)
    apples_bought = total_cost / apple_cost
    apples_total = apples_initial + apples_bought
    result = apples_total
    return result

> Finished chain.


'13.0'

# API Chains - OpenMeteo - Weather Information

In [50]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain

In [56]:
from langchain.chains.api import open_meteo_docs
weather_chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose = True,
    limit_to_domains = None
)

In [61]:
weather_chain.run("What is the temperature in the Mumbai.")



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=19.0760&longitude=72.8777&hourly=temperature_2m&temperature_unit=celsius
{"latitude":19.125,"longitude":72.875,"generationtime_ms":0.016927719116210938,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":6.0,"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-11-14T00:00","2023-11-14T01:00","2023-11-14T02:00","2023-11-14T03:00","2023-11-14T04:00","2023-11-14T05:00","2023-11-14T06:00","2023-11-14T07:00","2023-11-14T08:00","2023-11-14T09:00","2023-11-14T10:00","2023-11-14T11:00","2023-11-14T12:00","2023-11-14T13:00","2023-11-14T14:00","2023-11-14T15:00","2023-11-14T16:00","2023-11-14T17:00","2023-11-14T18:00","2023-11-14T19:00","2023-11-14T20:00","2023-11-14T21:00","2023-11-14T22:00","2023-11-14T23:00","2023-11-15T00:00","2023-11-15T01:00","2023-11-15T02:00","2023-11-15T03:00","2023-11-15T04:00","2023-11-15T05:00","2023-11-15T06:00","2023-11-15T07:0

' The temperature in Mumbai is expected to range from 24.1°C to 35.2°C over the next 7 days.'

In [62]:
weather_chain.run('What is the temperature like right now in Bedok, Singapore in degrees Celcius?')



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=1.3&longitude=103.9&hourly=temperature_2m&current_weather=true&temperature_unit=celsius
{"latitude":1.375,"longitude":103.875,"generationtime_ms":0.08606910705566406,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":6.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2023-11-14T19:45","interval":900,"temperature":25.9,"windspeed":7.1,"winddirection":285,"is_day":0,"weathercode":3},"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-11-14T00:00","2023-11-14T01:00","2023-11-14T02:00","2023-11-14T03:00","2023-11-14T04:00","2023-11-14T05:00","2023-11-14T06:00","2023-11-14T07:00","2023-11-14T08:00","2023-11-14T09:00","2023-11-14T10:00","2023-11-14T11:00","2023-11-14T12:00","2023-11-14T13:00","2023-11-14T14:00","202

' The temperature right now in Bedok, Singapore is 25.9°C.'

In [63]:
weather_chain.run("Is it cold in Varanasi?")



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=25.3176&longitude=82.9739&hourly=temperature_2m&temperature_unit=celsius
{"latitude":25.375,"longitude":83.0,"generationtime_ms":0.0209808349609375,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":81.0,"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-11-14T00:00","2023-11-14T01:00","2023-11-14T02:00","2023-11-14T03:00","2023-11-14T04:00","2023-11-14T05:00","2023-11-14T06:00","2023-11-14T07:00","2023-11-14T08:00","2023-11-14T09:00","2023-11-14T10:00","2023-11-14T11:00","2023-11-14T12:00","2023-11-14T13:00","2023-11-14T14:00","2023-11-14T15:00","2023-11-14T16:00","2023-11-14T17:00","2023-11-14T18:00","2023-11-14T19:00","2023-11-14T20:00","2023-11-14T21:00","2023-11-14T22:00","2023-11-14T23:00","2023-11-15T00:00","2023-11-15T01:00","2023-11-15T02:00","2023-11-15T03:00","2023-11-15T04:00","2023-11-15T05:00","2023-11-15T06:00","2023-11-15T07:00",

' The temperature in Varanasi is expected to be around 16-30°C over the next 7 days.'